In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn as nn
from torchvision import models
from tqdm import tqdm, trange
import time

In [2]:
# Define transforms
transform = transforms.Compose([
    transforms.Resize((300, 250)), # Resize images
    transforms.ToTensor(),               
])

# Load datasets
train_dataset = datasets.ImageFolder(
    root="Data/data1a/training",
    transform=transform
)

val_dataset = datasets.ImageFolder(
    root="Data/data1a/validation",
    transform=transform
)

# Create DataLoaders
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=2
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2
)

# debugging 
# print("Classes:", train_dataset.classes)
# print("Train batches:", len(train_loader))
# print("Val batches:", len(val_loader))

In [3]:
class ResNetBinary(nn.Module):
    def __init__(self):
        super().__init__()

        # Load ResNet18 model
        self.model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        num_features = self.model.fc.in_features 
        self.model.fc = nn.Linear(num_features, 2)

    def forward(self, x):
        return self.model(x)


In [4]:
# Checking for cuda
print("torch.version:", torch.version)
print("torch.version.cuda:", torch.version.cuda)
print("cuda available:", torch.cuda.is_available())
print("cuda device count:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("current device index:", torch.cuda.current_device())
    try:
        print("device name:", torch.cuda.get_device_name(0))
    except Exception as e:
        print("get_device_name failed:", e)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

torch.version: <module 'torch.version' from 'c:\\Users\\Saksham\\anaconda3\\Lib\\site-packages\\torch\\version.py'>
torch.version.cuda: 12.8
cuda available: True
cuda device count: 1
current device index: 0
device name: NVIDIA GeForce RTX 3050 Ti Laptop GPU
Using device: cuda


In [5]:
def train(model, dataset, epochs):
    optimizer = torch.optim.Adam(model.parameters())
    loss = nn.CrossEntropyLoss()
    dataloader = train_loader
    model = model.to(device)
    for epoch in trange(epochs):
        start = time.time()
        for (xs, targets) in tqdm(dataloader):
            xs, targets = xs.to(device), targets.to(device)
            ys = model(xs)
            optimizer.zero_grad()
            l = loss(ys, targets)
            l.backward()
            optimizer.step()
            with torch.no_grad():
                acc = (ys.argmax(axis=1) == targets).sum() / xs.shape[0]
        duration = time.time() - start
        print("[%d] acc = %.2f loss = %.4f in %.2f seconds." % (epoch, acc.item(), l.item(), duration))

In [6]:
model = ResNetBinary()

train(model, train_dataset, epochs=10)

 10%|█         | 1/10 [00:22<03:18, 22.08s/it]

[0] acc = 0.75 loss = 0.3818 in 22.07 seconds.


 20%|██        | 2/10 [00:36<02:18, 17.31s/it]

[1] acc = 0.81 loss = 0.3737 in 13.97 seconds.


 30%|███       | 3/10 [00:50<01:50, 15.78s/it]

...

[7] acc = 1.00 loss = 0.0077 in 14.01 seconds.


 90%|█████████ | 9/10 [02:13<00:14, 14.13s/it]

[8] acc = 0.88 loss = 0.2366 in 13.98 seconds.


100%|██████████| 10/10 [02:27<00:00, 14.75s/it]

[9] acc = 0.94 loss = 0.1739 in 13.71 seconds.


In [7]:
def validate(model, dataloader):
    """Run model on validation DataLoader and return loss/accuracy."""
    model = model.to(device)
    model.eval()
    loss_fn = nn.CrossEntropyLoss()
    total = 0
    correct = 0
    total_loss = 0.0
    with torch.no_grad():
        for xs, targets in tqdm(dataloader):
            xs = xs.to(device, non_blocking=True)
            targets = targets.to(device, non_blocking=True)
            ys = model(xs)
            loss = loss_fn(ys, targets)
            total_loss += loss.item() * xs.size(0)
            preds = ys.argmax(dim=1)
            correct += (preds == targets).sum().item()
            total += xs.size(0)
    avg_loss = total_loss / total if total > 0 else 0.0
    acc = correct / total if total > 0 else 0.0
    print(f"Validation - loss: {avg_loss:.4f}, accuracy: {acc:.4f} ({correct}/{total})")
    return {'loss': avg_loss, 'accuracy': acc, 'correct': correct, 'total': total}

In [8]:
try:
    _ = validate(model, val_loader)
except NameError:
    print('Validation DataLoader not found.')

100%|██████████| 15/15 [00:06<00:00,  2.21it/s]

Validation - loss: 0.5509, accuracy: 0.8565 (394/460)


In [ ]:
# Saving model weights
torch.save(model.state_dict(), "trained_weights/model.pth")